In [1]:
import os
# Change to the macrohive root directory
os.chdir('/Users/alighazizadeh/Documents/GitHub/mlops-zoomcamp')
!pwd

/Users/alighazizadeh/Documents/GitHub/mlops-zoomcamp


In [2]:
import torch
import numpy as np
import pandas as pd

import sklearn
import matplotlib
#from src.data.dataset import MotorImageryDataset

import numpy as np

from datetime import datetime
import importlib

# Train linear regression model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

%matplotlib inline
import matplotlib.pyplot as plt
# from sklearn.metrics import r2_score
import torch.nn.functional as F
import math
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler

from collections import OrderedDict
from sklearn.preprocessing import StandardScaler
# Fit DictVectorizer
from sklearn.feature_extraction import DictVectorizer
# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
print(device)

mps


In [3]:
df=pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
print(f'number of columns: {len(df.columns)}')


number of columns: 19


In [4]:

trip_duration = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60
df['duration'] = trip_duration


time_filter = (df.tpep_pickup_datetime < '2024-01-31') & (df.tpep_dropoff_datetime > '2024-01-01')

df_time_filter = df[time_filter]
trip_durtaion_std_jan=df_time_filter['duration'].std()
print(f'trip_durtaion_std_jan: {trip_durtaion_std_jan:.2f}')

trip_durtaion_std_jan: 34.83


In [5]:
duration_filter = (df['duration'] >= 1) & (df['duration'] <= 60)
df_time_duration_filter = df[time_filter & duration_filter]
df_time_duration_filter.shape

print(f'original rows: {len(df)}')
print(f'rows after time filter: {len(df_time_filter)}')
print(f'rows after time and duration filter: {len(df_time_duration_filter)}')
print(f'pct rows kept: {(len(df_time_duration_filter)/len(df_time_filter))*100:.2f}%')



original rows: 2964624
rows after time filter: 2862493
rows after time and duration filter: 2798843
pct rows kept: 97.78%


In [6]:
# Convert location IDs to strings and create list of dicts
dicts = df_time_duration_filter[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')
dv = DictVectorizer()
X = dv.fit_transform(dicts)
print(len(df_time_duration_filter))
print(f"number of columns of feature matrix: {X.shape[1]} should be equal to unique number of location ids {len(df_time_duration_filter['PULocationID'].unique())} + {len(df_time_duration_filter['DOLocationID'].unique())} ")


2798843
number of columns of feature matrix: 518 should be equal to unique number of location ids 257 + 261 


In [7]:
#  T
# Get target variable
y = df_time_duration_filter['duration'].values

# Initialize and train model
lr = LinearRegression()
lr.fit(X, y)

# Make predictions on training data
y_pred = lr.predict(X)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f'RMSE on training data: {rmse:.2f}')


RMSE on training data: 7.94


In [8]:
# Test on validation data from Feb 2024
test_time_filter = (df.tpep_pickup_datetime < '2024-02-28') & (df.tpep_dropoff_datetime > '2024-02-01')
df_test_time_duration_filter = df[test_time_filter & duration_filter]


# Get unique location IDs from training and test data
train_pu_locations = set(df_time_duration_filter['PULocationID'].unique())
train_do_locations = set(df_time_duration_filter['DOLocationID'].unique())
test_pu_locations = set(df_test_time_duration_filter['PULocationID'].unique())
test_do_locations = set(df_test_time_duration_filter['DOLocationID'].unique())

# Find intersections
pu_intersection = train_pu_locations.intersection(test_pu_locations)
do_intersection = train_do_locations.intersection(test_do_locations)

print(f"Number of PULocationID intersections: {len(pu_intersection)}")
print(f"Number of DOLocationID intersections: {len(do_intersection)}")
print(f" number of unique PULocationID in test data: {len(test_pu_locations)}")
print(f" number of unique DOLocationID in test data: {len(test_do_locations)}")

# Check if all test locations are in train locations
test_only_pu = test_pu_locations - train_pu_locations
test_only_do = test_do_locations - train_do_locations

print("\nPULocationIDs in test but not in train:", len(test_only_pu))
if len(test_only_pu) > 0:
    print("PULocationIDs:", sorted(test_only_pu))

print("\nDOLocationIDs in test but not in train:", len(test_only_do)) 
if len(test_only_do) > 0:
    print("DOLocationIDs:", sorted(test_only_do))


if len(test_only_pu) == 0 & len(test_only_do) == 0:
    print("All test locations are in train locations")
else:
    print("Some test locations are not in train locations")


Number of PULocationID intersections: 54
Number of DOLocationID intersections: 118
 number of unique PULocationID in test data: 54
 number of unique DOLocationID in test data: 118

PULocationIDs in test but not in train: 0

DOLocationIDs in test but not in train: 0
All test locations are in train locations


In [9]:
# As precaution, let's use the same locations in test as in training
train_locations = set(df_time_duration_filter['PULocationID'].unique()) | set(df_time_duration_filter['DOLocationID'].unique())

# Filter test data to only include locations that were in training data
valid_locations_mask = (
    df_test_time_duration_filter['PULocationID'].isin(train_locations) & 
    df_test_time_duration_filter['DOLocationID'].isin(train_locations)
)

df_test_time_duration_ID_filter = df_test_time_duration_filter[valid_locations_mask].copy()

# Create a DictVectorizer with all possible combinations from training data
dv = DictVectorizer(sparse=True)

# Fit DictVectorizer on training data only
train_dicts = df_time_duration_filter[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')
X = dv.fit_transform(train_dicts)

# Transform test data using the fitted DictVectorizer
test_dicts = df_test_time_duration_ID_filter[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')
X_test = dv.transform(test_dicts)

print(f"Training features: {X.shape[1]}")
print(f"Test features: {X_test.shape[1]}")

# Retrain the model with the new features
lr = LinearRegression()
y = df_time_duration_filter['duration'].values
lr.fit(X, y)
y_pred = lr.predict(X)
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f'RMSE on training data: {rmse:.2f}')

# Now predict
y_test = df_test_time_duration_ID_filter['duration'].values
y_pred_test = lr.predict(X_test)

rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f'RMSE on test data: {rmse_test:.2f}')

Training features: 518
Test features: 518
RMSE on training data: 7.94
RMSE on test data: 9.48
